In [ ]:
import pytorch_lightning as pl
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer


In [ ]:
# Definieer hulpfuncties
def load_data(file_path):
    """Laad een Excel-bestand en retourneer als pandas DataFrame."""
    return pd.read_excel(file_path)
def select_variables(dd):
    """Selecteer variabelen die moeten worden meegenomen."""
    return dd[(dd['Meenemen (0=nee, 1= ja, 2=onbekend, 3=computatie)'] == 1) & 
              (dd['0=input, 1=output, 2=onbekend'] != 2)]['Variable name'].tolist()


In [ ]:
# Definieer de PORSCH2DataModule klasse
class PORSCH2DataModule(pl.LightningDataModule):
    def __init__(self, data_files, dd_path, batch_size=32, missing_threshold=50):
        super().__init__()
        self.data_files = data_files
        self.dd_path = dd_path
        self.batch_size = batch_size
        self.missing_threshold = missing_threshold
            def prepare_data(self):
        # Implementatie van prepare_data methode
        # ... (code van eerdere implementatie)

    def setup(self, stage=None):
        # Implementatie van setup methode
        # ... (code van eerdere implementatie)

    def train_dataloader(self):
        train_dataset = TensorDataset(self.X_train, self.y_train)
        return DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        val_dataset = TensorDataset(self.X_val, self.y_val)
        return DataLoader(val_dataset, batch_size=self.batch_size)

In [ ]:
# Initialiseer de DataModule
data_files = ['path_to_file1.xlsx', 'path_to_file2.xlsx', ...]  # Voeg hier alle bestandspaden toe
dd_path = 'path_to_data_dictionary.xlsx'

data_module = PORSCH2DataModule(data_files, dd_path)

In [ ]:
# Voer data voorbereiding uit
data_module.prepare_data()

In [ ]:
#Controleer missende waarden (optioneel)
missing_data = data_module.data.isnull().sum() / len(data_module.data) * 100
print("Percentage missende waarden per variabele:")
print(missing_data.sort_values(ascending=False))

In [ ]:
#Set up de data voor training
data_module.setup()

In [ ]:
#Controleer de vorm van de trainingsdata (optioneel)
print("Vorm van trainingsdata:", data_module.X_train.shape)
print("Vorm van trainingslabels:", data_module.y_train.shape)

In [ ]:
#Definieer het model (voorbeeld)
class PORSCH2Model(pl.LightningModule):
    def __init__(self, input_size):
        super().__init__()
        self.layer1 = torch.nn.Linear(input_size, 64)
        self.layer2 = torch.nn.Linear(64, 32)
        self.layer3 = torch.nn.Linear(32, 1)

    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        return torch.sigmoid(self.layer3(x))

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = torch.nn.functional.binary_cross_entropy(y_hat, y)
        self.log('train_loss', loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.02)

In [ ]:
#Initialiseer het model
model = PORSCH2Model(input_size=data_module.X_train.shape[1])

In [ ]:
# Train het model
trainer = pl.Trainer(max_epochs=10)
trainer.fit(model, data_module)

In [ ]:
# Evalueer het model (voorbeeld)
model.eval()
with torch.no_grad():
    val_predictions = model(data_module.X_val)
    # Voeg hier evaluatiemetrieken toe